## Bayesian Network

In [ ]:
import pandas as pd 
import networkx as nx 
import matplotlib.pyplot as plt 

In [ ]:
df = pd.read_csv('BBN_data.csv')
df = df.drop(["Unnamed: 0"], axis=1)
df

## Structural Learning

### Search strategy

The search space of DAG is exponential in the number of variables, thus exhaustive search is unaffordable for big networks; moreover, the scoring function allows for local maxima, which prevents local optimization algorithms to always find the optimal structure. 

Thus, the following approach, which takes advantage of Exhaustive Search, is only reccomended for small networks (less than 5 nodes), as the one considered here; once more nodes are involved, one may want to implement a Hill Climb Search.

The scoring function chosen is BDeu (Bayesian Dirichlet equivalent uniform prior), which is a popular and reasonable choice.

In [ ]:
from pgmpy.estimators import MmhcEstimator
from pgmpy.estimators import HillClimbSearch, ExhaustiveSearch, BDeuScore

est = ExhaustiveSearch(df, scoring_method=BDeuScore(df))
model = est.estimate()
print("Model:    ", model.edges())

## Parameter Learning

In [ ]:
# conda install -c ankurankan pgmpy
from pgmpy.models import BayesianModel, BayesianNetwork
from pgmpy.estimators import ParameterEstimator

In [ ]:
model = BayesianNetwork([('Num_of_people', 'time'), ('mood', 'Num_of_people'), ('mood', 'place'), ('mood', 'time'), 
                         ('place', 'Num_of_people'), ('place', 'time'), ('what', 'Num_of_people'), ('what', 'mood'), 
                         ('what', 'place'), ('what', 'time')])

In [ ]:
# PROBABILITIES
# using MLE
from pgmpy.estimators import MaximumLikelihoodEstimator
mle = MaximumLikelihoodEstimator(model, df)
print(mle.estimate_cpd('mood'))  # conditional probabilities
print(mle.estimate_cpd('what'))  # unconditional probabilities

As it is possible to see, the maximum likelihood approach is not the best as it would put to zero, i.e. impossible configuration, the probabilities of configurations which were never observed in the data. 

Given that it is quite likely that some configurations were never observed in the two weeks of data collections, while also being totally possible outcomes, it is more useful to use Bayesian Parameters Estimation, to assign priors probabilities and prevent overfitting the data. 

The Bayesian Parameter Estimator starts with prior conditional probability distribution (CPD), that express our beliefs about the variables before the data are observed. Those priors are then updated, using the observed data.
A common choice for the priors is the BDeu (Bayesian Dirichlet equivalent uniform prior).

In [ ]:
from pgmpy.estimators import BayesianEstimator

# est = BayesianEstimator(model, df)
# print(est.estimate_cpd('what', prior_type='BDeu', equivalent_sample_size=10))

model.fit(df, estimator=BayesianEstimator, prior_type="BDeu", equivalent_sample_size=10)
for cpd in model.get_cpds():
    print(cpd)

In [ ]:
# check for errors:
# - Checks if the sum of the probabilities for each state is equal to 1 (tol=0.01).
# - Checks if the CPDs associated with nodes are consistent with their parents.
# returns True if all checks are passed; False if not
model.check_model()

In [ ]:
# local independences
# print(model.local_independencies('Num_of_people'))
# print(model.local_independencies('time'))
# print(model.local_independencies('mood'))
# print(model.local_independencies('what'))
# print(model.local_independencies('place'))

# Getting all the local independencies in the network at once
print("All local independencies in the network:")
model.local_independencies(['what', 'mood', 'Num_of_people', 'place', 'time'])

In a Bayesian Network, generally we would like to encode as much local independencies as possible, because this simplify a lot computations and it is one of the main advatage of BN. 

However, in the considered case, having a small and easy network, and having as previous knowledge the fact that all factors involved have some level of influence on the others, we can accept a fully connected network as a good choice, as it would allow us to see the effects of all factors on the others, which is the primary goal here.

In [ ]:
# Active trail:
# For any two variables A and B in a network if any change in A influences the values of B 
# then we say that there is an active trail between A and B.
# In pgmpy, active_trail_nodes returns the set of the nodes affected by any change of the node passed as argument.
print("Active trail for activity:")
print(model.active_trail_nodes('what'))
print("Active trail for activity when mood is observed:")
print(model.active_trail_nodes('what', observed='mood')) # seems like mood influence the activity done

In [ ]:
# represent network
nx.draw(model, with_labels = True, node_size=1000, font_size=10)
plt.figure(1,figsize=(12,12)) 
plt.show()

In [ ]:
from pgmpy.inference import VariableElimination

infer = VariableElimination(model)
print('Variable Elimination:')
print(infer.query(['what']))
print(infer.query(['what'], evidence={'mood': 3.0, 'Num_of_people': '1 - 3'}))

In [ ]:
print(infer.map_query(['what'])) 
# return the activity with the highest probability among all of them  
# when there is no evidence on others variables,
# thus, simply the one with highest probability on the "what" table

In [ ]:
# returns the activity having the highest probability to happen, given than the mood and the number of people are observed
# here is a neutral mood and more than 4 people around (high number of people)
print(infer.map_query(['what'], evidence={'mood': 2.0, 'Num_of_people': '>= 4'}))

In [ ]:
# same as before but changing the mood in a positive one, keeping same number of people around
print(infer.map_query(['what'], evidence={'mood': 4.0, 'Num_of_people': '>= 4'}))

In [ ]:
print(infer.map_query(['what'], evidence={'mood': 3.0, 'Num_of_people': '>= 4', 'time': 'Afternoon', 
                                          'place': 'Outdoor'}))

In [ ]:
print(infer.query(['what'], evidence={'Num_of_people': '>= 4', 'time': 'Afternoon'}))

In [ ]:
print(infer.query(['what'], evidence={'Num_of_people': '1 - 3', 'time': 'Morning'}))

## Checking activity predictions precision on a test set given evidence on all other variables

In [ ]:
import pandas as pd 

from pgmpy.estimators import MmhcEstimator
from pgmpy.estimators import HillClimbSearch, ExhaustiveSearch, BDeuScore

from pgmpy.models import BayesianModel, BayesianNetwork
from pgmpy.estimators import ParameterEstimator

from pgmpy.estimators import BayesianEstimator

from pgmpy.inference import VariableElimination

import networkx as nx
import matplotlib.pyplot as plt 

In [ ]:
df = pd.read_csv('BBN_data.csv')
df = df.sample(frac=1, random_state=1) # shuffle

# split into test and train set
train = df[:len(df)//2]
test = df[len(df)//2:]
train = train.drop(["Unnamed: 0"], axis=1)
test = test.drop(["Unnamed: 0"], axis=1)
train = pd.DataFrame(train)


Find optimal structure on training set.

In [ ]:
est = ExhaustiveSearch(train, scoring_method=BDeuScore(train))
model = est.estimate()
print("Model:    ", model.edges())

In [ ]:
model = BayesianNetwork([('Num_of_people', 'mood'), ('Num_of_people', 'place'), ('place', 'mood'), 
                         ('time', 'Num_of_people'), ('time', 'mood'), ('time', 'place'), ('what', 'Num_of_people'), 
                         ('what', 'mood'), ('what', 'place'), ('what', 'time')])

Fit the model on training set.

In [ ]:
model.fit(train, estimator=BayesianEstimator, prior_type="BDeu", equivalent_sample_size=10)

In [ ]:
model.check_model() # if True, no error

print("All local independencies in the network:\n")
model.local_independencies(['what', 'mood', 'Num_of_people', 'place', 'time'])

print("Active trail for activity:")
print(model.active_trail_nodes('what'))
print(model.active_trail_nodes('Num_of_people'))
print(model.active_trail_nodes('time'))
print(model.active_trail_nodes('place'))
print(model.active_trail_nodes('mood'))

In [ ]:
nx.draw(model, with_labels = True, node_size=1000, font_size=10)
plt.figure(1,figsize=(12,12)) 
plt.show()

Check accuracy of prediction when all evidence is given, on test set.

In [ ]:
infer = VariableElimination(model)

In [ ]:
test = test.reset_index()
test = test.drop(["index"], axis=1)
test.mood.astype('float32').dtypes

In [ ]:
correct = 0
wrong = 0

for row in range(len(test)):
    x = {}
    x['mood'] = test.mood[row]
    x['place'] = test.place[row]
    x['Num_of_people'] = test.Num_of_people[row]
    x['time'] = test.time[row]
    y = test.what[row]
    print(x)
    l = infer.map_query(['what'], evidence = dict(x))
    print(l['what'], '= ?', y)
    if l['what'] == y:
        correct +=1
    else:
        wrong += 1

In [ ]:
print(wrong, correct)

In [ ]:
accuracy = correct / (wrong+correct) * 100
print(str(round(accuracy, 1)), '%')